# Assignment 1: Computing the Area of the Mandelbrot Set

Chayenne Olumuyiwa (12055662) & Liesbet Ooghe (13755471)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from scipy.stats import qmc

In this notebook we are exploring the Mandelbrot set, and we estimate its area using a Monte Carlo integration. We will do this by visualizing the interesting set of numbers, and perform a Monte Carlo approach using three different sampling techniques.

In [ ]:
def converges(a0, b0, iterations):
    """
    Defines if the point (a0, b0) converges following the Mandelbrot Iteration (Z_{n+1} = Z_n ^2 + c) after iterations iterations
    """
    a = a0
    b = b0

    for i in range(iterations):
        a_temporary = a
        a = (a**2 - b**2) + a0
        b = (2 * a_temporary * b) + b0

        if abs(a) > 100 and abs(b) > 100:
            return False
        
    return True

In [ ]:
def calculate_area(Samples, Iterations):
    a = np.array([coord[0] for coord in Samples])
    b = np.array([coord[1] for coord in Samples])
    
    converges_results = np.array([converges(a0, b0, Iterations) for a0, b0 in zip(a, b)])
    N_converges = np.sum(converges_results)
    
    area = N_converges / len(a) * 9  # Scale by the area of the space you're sampling

    return area

In order to estimate the area, we will use different forms of sampling. Pure random sampling, latin hypercube sampling, and orthogonal sampling.


In [ ]:
def pure_random(n):
    return [[random.random()*3-2, random.random()*3-1.5] for x in range(n)]

In [ ]:
def latin_hypercube(n):
    LHC_object = qmc.LatinHypercube(2)
    LHC = LHC_object.random(n)

    for j in range(len(LHC)):
        LHC[j, 0] = LHC[j, 0] * 3 - 2
        LHC[j, 1] = LHC[j, 1] * 3 - 1.5

    return LHC

In [ ]:
def orthogonal_sampling(Accuracy, n):
    """
    Divides grid into Accuracy * Accuracy subsquares, then takes evenly orthogonal samples from each subsquare.
    """
    samples = []

    #Map total grid
    x_intervals = np.linspace(-1, 2, Accuracy + 1)   # Later identified as faulty
    y_intervals = np.linspace(-1.5, 1.5, Accuracy + 1) 

    #Find number of samples per grid square
    grid_size = Accuracy ** 2
    if n % grid_size == 0:
        samples_per_square = int(n / grid_size)
    else:
        # Round up the amount of samples per square if it results in a float
        samples_per_square = math.ceil(n / grid_size)  
        
    LHC_object = qmc.LatinHypercube(d=2) 

    #Perform hypercube sampling in eaach grid
    for i in range(Accuracy):
        for j in range(Accuracy):
            x_start, x_end = x_intervals[i], x_intervals[i + 1]
            y_start, y_end = y_intervals[j], y_intervals[j + 1]

            LHC = LHC_object.random(samples_per_square)

            for k in range(samples_per_square):
                x_sample = LHC[k, 0] * (x_end - x_start) + x_start
                y_sample = LHC[k, 1] * (y_end - y_start) + y_start

                samples.append([x_sample, y_sample])

    return samples

First we will investigate what happens if we keep i constant (10 000), and start increasing the sample size from 1 to 500

In [ ]:
# First plot where i is set, s varies
s_range = 500
iterations = 10000
repetitions = 100

areas_random = []
areas_lhc = []
areas_orthogonal = []

for n in range(1, s_range):
    random_areas = []
    lhc_areas = []
    orthogonal_areas = []

    for rep in range(repetitions):
        samples_random = pure_random(n)
        samples_lhc = latin_hypercube(n)
        samples_orthogonal = orthogonal_sampling(4, n)
        
        area_random = calculate_area(samples_random, iterations)
        area_lhc = calculate_area(samples_lhc, iterations)
        area_orthogonal = calculate_area(samples_orthogonal, iterations)
        
        random_areas.append(area_random)
        lhc_areas.append(area_lhc)
        orthogonal_areas.append(area_orthogonal)

    mean_random = np.mean(random_areas)
    mean_lhc = np.mean(lhc_areas)
    mean_orthogonal = np.mean(orthogonal_areas)
    
    ci_random = 1.96 * np.std(random_areas) / np.sqrt(repetitions)
    ci_lhc = 1.96 * np.std(lhc_areas) / np.sqrt(repetitions)
    ci_orthogonal = 1.96 * np.std(orthogonal_areas) / np.sqrt(repetitions)

    areas_random.append((mean_random, ci_random))
    areas_lhc.append((mean_lhc, ci_lhc))
    areas_orthogonal.append((mean_orthogonal, ci_orthogonal))

means_random, cis_random = zip(*areas_random)
means_lhc, cis_lhc = zip(*areas_lhc)
means_orthogonal, cis_orthogonal = zip(*areas_orthogonal)

plt.figure(figsize=(8, 6))

plt.fill_between(range(1, s_range), np.array(means_random) - np.array(cis_random), 
                 np.array(means_random) + np.array(cis_random), color='orange', alpha=0.3)
plt.fill_between(range(1, s_range), np.array(means_lhc) - np.array(cis_lhc), 
                 np.array(means_lhc) + np.array(cis_lhc), color='purple', alpha=0.3)
plt.fill_between(range(1, s_range), np.array(means_orthogonal) - np.array(cis_orthogonal), 
                 np.array(means_orthogonal) + np.array(cis_orthogonal), color='blue', alpha=0.3)

plt.plot(range(1, s_range), means_random, label='Random Sampling', color='orange', linewidth=2)
plt.plot(range(1, s_range), means_lhc, label='Latin Hypercube', color='purple', linewidth=2)
plt.plot(range(1, s_range), means_orthogonal, label='Orthogonal Sampling', color='blue', linewidth=2)
plt.ylim(0.5, 2.5) 

plt.xlabel('Number of Samples (s)', fontsize=12)
plt.ylabel('Calculated Area', fontsize=12)
plt.title('Estimated Area of the Mandelbrot Set (with Confidence Interval)', fontsize=14)
plt.legend()

plt.figtext(0.5, 0.01, 'For n_iteration = 10,000, and n_runs = 100', ha='center', va='bottom', fontsize=10)
plt.show()
print(f"random value is {means_random[-1], cis_random[-1]}, lhc value is {means_lhc[-1], cis_lhc[-1]}, orthogonal value is {means_orthogonal[-1], cis_orthogonal[-1]}")

Then we will set the amount of samples taken to 10 000, and vary the amount of iterations

In [ ]:
# Then plot where s is set, i varies
s = 10000
iterations_range = 500
repetitions = 100

areas_random = []
areas_lhc = []
areas_orthogonal = []

for it in range(1,iterations_range):
    random_areas = []
    lhc_areas = []
    orthogonal_areas = []
    
    for rep in range(100):
        samples_random = pure_random(s)
        samples_lhc = latin_hypercube(s)
        samples_orthogonal = orthogonal_sampling (4, s)
        
        area_random = calculate_area(samples_random, it)
        area_lhc = calculate_area(samples_lhc, it)
        area_orthogonal = calculate_area(samples_orthogonal, it)
        
        random_areas.append(area_random)
        lhc_areas.append(area_lhc)
        orthogonal_areas.append(area_orthogonal)

    mean_random = np.mean(random_areas)
    mean_lhc = np.mean(lhc_areas)
    mean_orthogonal = np.mean(orthogonal_areas)
    
    ci_random = 1.96 * np.std(random_areas) / np.sqrt(repetitions)
    ci_lhc = 1.96 * np.std(lhc_areas) / np.sqrt(repetitions)
    ci_orthogonal = 1.96 * np.std(orthogonal_areas) / np.sqrt(repetitions)

    areas_random.append((mean_random, ci_random))
    areas_lhc.append((mean_lhc, ci_lhc))
    areas_orthogonal.append((mean_orthogonal, ci_orthogonal))

means_random, cis_random = zip(*areas_random)
means_lhc, cis_lhc = zip(*areas_lhc)
means_orthogonal, cis_orthogonal = zip(*areas_orthogonal)
        
plt.figure(figsize=(8, 6))
plt.fill_between(range(0, iterations_range), np.array(means_random) - np.array(cis_random), 
                 np.array(means_random) + np.array(cis_random), color='orange', alpha=0.3)
plt.fill_between(range(0, iterations_range), np.array(means_lhc) - np.array(cis_lhc), 
                 np.array(means_lhc) + np.array(cis_lhc), color='purple', alpha=0.3)
plt.fill_between(range(0, iterations_range), np.array(means_orthogonal) - np.array(cis_orthogonal), 
                 np.array(means_orthogonal) + np.array(cis_orthogonal), color='blue', alpha=0.3)
        
plt.plot(range(0, iterations_range), means_random, label='Random Sampling', color='orange', linewidth=2)
plt.plot(range(0, iterations_range), means_lhc, label='Latin Hypercube', color='purple', linewidth=2)
plt.plot(range(0, iterations_range), means_orthogonal, label='Orthogonal Sampling', color='blue', linewidth=2)
plt.ylim(1.3, 1.65) 
        
plt.xlabel('Number of Iterations (i)', fontsize=12)
plt.ylabel('Calculated Area', fontsize=12)
plt.title('Estimated Area of the Mandelbrot Set (with Confidence Interval)', fontsize=14)
plt.legend()

plt.figtext(0.5, 0.01, "For n_samples = 10,000, and n_runs = 100", ha='center', va='bottom', fontsize=10)
plt.show        
print(f"random value is {means_random[-1], cis_random[-1]}, lhc value is {means_lhc[-1], cis_lhc[-1]}, orthogonal value is {means_orthogonal[-1], cis_orthogonal[-1]}")

We want to make some nice images of the Mandelbrot set for thr introduction.

In [ ]:
def mandelbrot(list1):
    Lx_blue, Ly_blue = [], []

    for item in list1:
        a0 = item[0]
        b0 = item[1]

        con = converges(a0, b0, 10000)

        if con == True:
            Lx_blue.append(a0)
            Ly_blue.append(b0)

    return Lx_blue, Ly_blue

In [ ]:
points = pure_random(1000000)
MS_x, MS_y = mandelbrot(points)
plt.plot(MS_x, MS_y, "ko", markersize = 0.5)
plt.xlim(-2, 1)
plt.ylim(-1.5, -1.5)
plt.axis('square')
plt.axis('off')
plt.show()

In [ ]:
points_zoom = [[random.random()*3-2, random.random()*3-1.5] for _ in range(1000000)]
MS_x_zoom, MS_y_zoom = mandelbrot(points_zoom)
plt.plot(MS_x_zoom, MS_y_zoom, "ko", markersize = 0.5)
plt.xlim(-2, 1)
plt.ylim(-1.5, -1.5)
plt.axis('square')
plt.axis('off')
plt.show()

We will briefly look into the area calculation usong Sobol sequences instead of teh other three sampling methods.

In [ ]:
def sobol(m):
    Sob_object = qmc.Sobol(2)
    Sob = Sob_object.random_base2(m)

    for j in range(len(Sob)):
        Sob[j, 0] = Sob[j, 0] * 3 - 2
        Sob[j, 1] = Sob[j, 1] * 3 - 1.5
    
    return Sob

In [ ]:
areas_sobol = []

for n in range(1, 9):
    print("hi")
    sobol_areas = []

    for rep in range(100):
        samples_sobol = sobol(n)
        area_sobol = calculate_area(samples_sobol, 10000)
        sobol_areas.append(area_sobol)

    mean_sobol = np.mean(sobol_areas)
    ci_sobol = 1.96 * np.std(sobol_areas) / np.sqrt(100)
    areas_sobol.append((mean_sobol, ci_sobol))

means_sobol, cis_sobol = zip(*areas_sobol)

plt.figure(figsize=(8, 6))
plt.fill_between([2 ** x for x in range(1, 9)], np.array(means_sobol) - np.array(cis_sobol), 
                 np.array(means_sobol) + np.array(cis_sobol), color='orange', alpha=0.3)
plt.plot([2 ** x for x in range(1, 9)], means_sobol, label='Sobol Sequence', color='orange', linewidth=2)
plt.ylim(0.5, 2.5) 

plt.xlabel('Number of Samples (s)', fontsize=12)
plt.ylabel('Calculated Area', fontsize=12)
plt.ylim(1.3, 1.65)
plt.title('Estimated Area of the Mandelbrot Set (with Confidence Interval)', fontsize=14)
plt.legend()

plt.figtext(0.5, 0.01, 'For n_iteration = 10,000, and n_runs = 100', ha='center', va='bottom', fontsize=10)

plt.show()